# Analysis of Personal Finance Bookkeeping Activity
What is the motivation for conducting this analysis?

Exploring bookkeeping of personal income and spending can reveal that...

Users who enjoy tracking their expenses and income might be,
depending on their approach and enthusiasm for financial management.
described as:

- **Budget-conscious** – Someone who carefully plans and monitors their spending.
- **Financially meticulous** – A person who pays close attention to financial details.
- **Expense tracker** – Someone who actively records their spending habits.
- **Frugal planner** – A person who enjoys optimizing their finances for savings.
- **Money-savvy** – Someone who is knowledgeable and strategic about finances.
- **Personal finance enthusiast** – A broader term for those who enjoy managing their money.
- **Data-driven spender** – Someone who makes financial decisions based on recorded data.
- **Financial optimizer** – A person who seeks to maximize efficiency in their financial habits.

On the other hand, users who don’t track their expenses and income regularly might be,
depending on their habits and attitudes toward financial management, described as such:

- **Spontaneous spender** – Someone who makes purchases without much planning.
- **Financially carefree** – A person who doesn’t stress about tracking money closely.
- **Unstructured budgeter** – Someone who manages finances loosely without detailed records.
- **Impulse buyer** – A person who tends to make purchases on a whim.
- **Money-agnostic** – Someone who doesn’t prioritize financial tracking.
- **Casual earner** – A person who earns and spends without strict oversight.
- **Non-budgeter** – Someone who avoids formal budgeting altogether.
- **Financially intuitive** – A person who relies on instinct rather than detailed tracking.

# Data Source

- acc_user
- acc_cashflow

In [ ]:
import os
from sqlalchemy import create_engine
import pandas as pd

user = os.getenv("MYSQL_USER")
password = os.getenv("MYSQL_PASSWORD")
host = "localhost"
database = "zoo"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

## Users

In [ ]:
## user registration , i.e. account creation, date
## is_agree: is communication from official account blocked
## to be deprecated: registered

query = """
SELECT
  user_id,
  isBot is_bot,
  isAgree is_agree,
  min(CREDTM) registered,
  min(CREDTM) ts
FROM zoo_checkchick2.ACC_USER
GROUP BY user_id, is_bot, is_agree
"""

users = pd.read_sql(query, con=engine,
                    dtype=({'is_bot':'bool', 'is_agree':'bool', 'ts':'datetime64[ns]', 'registered':'datetime64[ns]'}))
users.info()

In [ ]:
# check for duplicates
x = users.groupby('user_id').size()
dups=x[x > 1].index
len(dups)

In [ ]:
users[users.user_id.isin(dups)].shape

In [ ]:
# what is dup ratio?
89192/44596

In [ ]:
users[users.user_id.isin(dups)].sort_values(['user_id', 'registered']).tail(10)

Remove duplicates by keeping the last row (most recent) in each group.


In [ ]:
users.sort_values(['user_id','registered'], inplace=True)

In [ ]:
x = users.groupby('user_id', as_index=False).last()
x[x.user_id.isin(dups)]

In [ ]:
x.shape

In [ ]:
users = x

In [ ]:
# how many Bots?
print(users.is_bot.sum())

In [ ]:
users.query("is_bot")

Impute `user_id`  
Real user_id starts with captial 'U'

In [ ]:
i = users.user_id.str.lower().str.startswith('u')
users[i].shape
users.loc[i, 'user_id'] = users[i]['user_id'].str.capitalize()

In [ ]:
cohorts = users.groupby(users.registered.dt.year).size()
cohorts.name = 'n_new_user'
_ = cohorts.plot.bar(rot=0, xlabel='cohort year', ylabel='newly registered users')

## Cashflow

In [ ]:
# acc_cashflow dataset
# check number of records (expense and income entries) each month

cashflow = pd.read_feather('../data/cashflow.feather')

df = cashflow \
    .groupby(cashflow.ts.dt.to_period('M')) \
    .agg(total = ('ts','size'),
         nbr_expense_entry = ('is_expense', 'sum'),
         nbr_group_entry = ('is_group', 'sum')
        )
_ = df.plot.line(y=['total', 'nbr_expense_entry', 'nbr_group_entry'], xlabel='')
print(df.describe())

In [ ]:
print(df[df.total > df.total.quantile(.51)])

__*Observation:*__
- errorenous timestamps
- personal entries makes up significally larger portions of records; I had expected more entries. 
- expenses makes up larger portion of the records; this aligns with the norm for personal finance dataset.

In [ ]:
# select date range
# between '2018-06-01' AND '2025-02-01'
# to exclude erroneous rows

tsl = pd.to_datetime(['2018-06-01', '2025-02-01'])
cashflow = cashflow.query("@tsl[0] <= ts < @tsl[1]").copy()
cashflow.info()

_**n.b.,**_ __amt__ is float64. _I expect this to be whole number._

In [ ]:
cashflow.isna().sum()

In [ ]:
print(cashflow.group_id.count() / cashflow.shape[0])

_**Observation:**_ group entries make up about 15% of the records.

_**Question:**_ Should factional amounts be rounded up?

In [ ]:
# should amt be whole number?
bad = cashflow.query("amt % 1 != 0")
bad.groupby('is_expense')['amt'].describe()

In [ ]:
bad['category'].unique()

In [ ]:
bad['note'].unique()

In [ ]:
# impute fractional amounts by rounding up if amt > 0 and down if amt < 0

cashflow['amt'] = cashflow['amt'].round().astype('Int64')

In [ ]:
cashflow.info()

In [ ]:
# Should amount equals 0 be excluded?

cashflow[cashflow.amt == 0].count()

In [ ]:
# check number of active users in each month, i.e. period

cashflow['yyyy_mm'] = cashflow['ts'].dt.to_period('M')
cashflow['yyyy'] = cashflow['ts'].dt.year
cashflow['mm'] = cashflow['ts'].dt.month

#cashflow.groupby(['yyyy', 'mm']).agg({'user_id':'nunique'}).unstack(level=1)

In [ ]:
_ = cashflow.groupby('yyyy_mm').agg({'user_id':'nunique'}) \
    .plot(y='user_id', kind='line', figsize=(12, 6), title="Number of Active Users Each Month")

__*Observation:*__ number of active users gradually declined but not obvious

In [ ]:
cashflow.describe()

Impute `group_id` is blank but not `None`

In [ ]:
bad = cashflow.group_id.str.strip().str.len() == 0  # group_id missing
bad.value_counts()

In [ ]:
cashflow.loc[bad, 'user_id'].nunique()

In [ ]:
# impute
cashflow.loc[bad, ['group_id', 'is_group']] = [None, False]

Erroneous `user_id`
Valid `user_id` must start with captial U

In [ ]:
cashflow['isBad'] = ~cashflow.user_id.str.lower().str.startswith('u')

In [ ]:
cashflow.groupby('isBad').size()

In [ ]:
cashflow[bad].describe()

Impute valid `user_id`  
Make sure they start with capital U

In [ ]:
cashflow.loc[~cashflow.isBad, 'user_id'] = cashflow[~cashflow.isBad]['user_id'].str.capitalize()

# Tidy Dataset

Let the final tidy dataset be `td`. Proceed with the data preparation as follow:

## Tenure

Tenure here is defined as to begin when user submit first entry,
and does not consider when user first followe dor added LINE OA
(offical account)

In [ ]:
# user tenure, income and expense entry stats,
# including group entries
# excluding amount = 0

tenure = cashflow[(cashflow.amt != 0) & ~cashflow.isBad] \
    .groupby('user_id') \
    .agg(user_tenure = ('ts', lambda x: x.max() - x.min()),
         first_entry = ('ts', 'min'),
         last_entry = ('ts', 'max'),
         nbr_entry = ('user_id', 'count'),
         total_exp = ('amt', lambda x: x[x < 0].sum()),
         nbr_exp = ('amt', lambda x: x[x < 0].count()),
         total_inc = ('amt', lambda x: x[x > 0].sum()),
         nbr_inc = ('amt', lambda x: x[x > 0].count())
        )
tenure.info()

In [ ]:
tenure.describe()

_**Initial observation:**_ Of the 397,208 users, 75% of which had **not** logged entries in the last 46~50 days 

In [ ]:
# WIP category count
td_cat = cashflow[~cashflow.isBad].groupby(['user_id', 'is_expense'])['category_id'].nunique().unstack(level=1, fill_value=0)
td_cat.describe()

_**Initial observation:**_ Is it true that entries are not well categorized by the bottom 75% of users?

## Group Bookkeeping

In [ ]:
# user group expense and income entry stat per user
# add count of categories

td_grp = cashflow[(cashflow.amt != 0) & cashflow.is_group & ~cashflow.isBad] \
    .groupby('user_id') \
    .agg(n_grp = ('group_id', 'nunique'),
         first_grp_entry = ('ts', 'min'),
         last_grp_entry = ('ts', 'max'),
         grp_exp = ('amt', lambda x: x[x < 0].sum()),
         nbr_grp_exp = ('amt', lambda x: x[x < 0].count()),
         grp_inc = ('amt', lambda x: x[x > 0].sum()),
         nbr_grp_inc = ('amt', lambda x: x[x > 0].count())
        )
td_grp.info()

In [ ]:
td_grp.describe()

## Co-bookkeepers

self-joined group_id:user_id from _cashflow_ to compute
number of distinct users interacted within groups

In [ ]:
# unique group_id:user_id linkage 
grp = cashflow.loc[(cashflow.amt != 0) & (cashflow.group_id.notnull()) & ~cashflow.isBad,
                 ['group_id', 'user_id']].drop_duplicates()
grp.info()

In [ ]:
# user's groups and their assoicated users (members),
# i.e., user's connections with other users thru cooperative bookkeeping 

mbr = grp[['user_id', 'group_id']].merge(grp[['user_id', 'group_id']], on='group_id', how='left')
mbr.columns = ['user_id', 'group_id', 'member_id']
mbr.describe()

In [ ]:
# count the participants at each group level for every user

x = mbr.groupby(['user_id', 'group_id']) \
    .agg({'member_id':'nunique'}) \
    .reset_index(1) \
    .rename(columns={'member_id':'nbr_member'})
x.describe()

In [ ]:
x.plot.hist(bins=30, alpha=0.7)

_**Observation:**_ Groups with only one participant should be excluded in the stat summary.

In [ ]:
# for every user, count the unique users across _all_ associated groups

cnx = mbr.groupby('user_id').agg({'group_id': 'nunique', 'member_id':'nunique'})
cnx.columns = ['n_grp', 'nbr_connection']
cnx['nbr_connection'] = cnx['nbr_connection'] - 1 # remove user itself from count
cnx.hist(bins=40, grid=False, alpha=.7)

## Merge and Impute

In [ ]:
td = users.merge(tenure, on='user_id', how='right') \
    .merge(cnx['nbr_connection'], on='user_id', how='left') \
    .merge(td_grp, on='user_id', how='left') \
    .convert_dtypes()
td.info()

In [ ]:
values = {'is_bot':False, 'is_agree':False,
          'nbr_connection':0, 'n_grp':0,
          'grp_exp':0, 'nbr_grp_exp':0,
          'grp_inc':0, 'nbr_grp_inc':0}
td.fillna(value=values, inplace=True)

In [ ]:
td.describe()

In [ ]:
td.info()

In [ ]:
# raise exception if file already exists but allow the run to proceed
td.to_csv('../reports/tidy.csv', mode='x')

# EDA

In [ ]:
td[td.is_bot]

_**Observation:**_ Nothing too interesting to note

In [ ]:
# dropping Bots from future analysis steps
td.drop(td[td.is_bot].index, axis=0, inplace=True)

## Frequency of Logging Financial Transactions

What is the typical frequency?  

In [ ]:
# set datetime index in order to resample frequency of event
cashflow.set_index('ts', inplace=True)

In [ ]:
weekly_counts = cashflow.groupby(['user_id', pd.Grouper(freq='W')])['user_id'].count()

In [ ]:
fq = weekly_counts.groupby('user_id').agg({'count', 'median', 'mean'})
fq.columns = ['nbr_wks', 'fq_median', 'fq_mean']

In [ ]:
fq.describe(percentiles=[.25, .5, .6, .7, .75, .8, .9, .95, .99])

In [ ]:
# however if we just want to know how frequent users records their personal finance,
# it is not important to separate income from expense entries

cashflow.reset_index(inplace=True)
y_sorted = cashflow.sort_values(['user_id', 'ts'])
y_sorted['days_elapsed'] = y_sorted.groupby('user_id')['ts'].diff()

In [ ]:
y_sorted[['user_id', 'ts', 'days_elapsed']].tail(30)

In [ ]:
# lapsed = y_sorted.groupby('user_id').agg({'days_elapsed':['median', 'mean', 'max']})
# n.b. a known bug with median, use quantile(0.5) workaround
lapsed = y_sorted.groupby('user_id').agg({'days_elapsed':[lambda x: x.quantile(0.5), 'mean', 'max']})
lapsed.describe()

In [ ]:
fq

### WIP

In [ ]:
# 
x_sorted = cashflow.sort_values(['user_id', 'is_expense', 'ts'])
x_sorted['days_elapsed'] = x_sorted.groupby(['user_id', 'is_expense'])['ts'].diff()

In [ ]:
x_sorted[['user_id', 'is_expense', 'ts', 'days_elapsed']].tail(30)

In [ ]:
fq_ = x_sorted.groupby(['user_id', 'is_expense']).agg({'days_elapsed':[lambda x: x.quantile(0.5), 'mean', 'max']})

In [ ]:
fq_.query("is_expense").describe()

In [ ]:
#fq =
fq_.groupby(['is_expense']).agg(['min', 'median', 'mean', 'max'])
# fq

In [ ]:
td[td.user_id=='Uffffed94576a41cb306b899c40719ed9']

In [ ]:
fq_.xs('Uffffed94576a41cb306b899c40719ed9', level='user_id')

In [ ]:
fq_.xs(False, level='is_expense').agg(['min','mean','median'])
# fq.index

## WIP: Group of One

What are the patterns of this cluster of users
who tracks income or expenses using one or more groups
with no other members in the group?

In [ ]:
x = mbr.groupby('group_id').agg(n_member=('member_id', 'nunique'))
y = x.query('n_member == 1')
grp_lst = [i for i in y.index]
grp_one = cashflow[(cashflow.amt != 0) & (cashflow.group_id.isin(grp_lst)) & ~cashflow.isBad] \
    .groupby('group_id') \
    .agg(first_entry=('ts', 'min'), last_entry=('ts', 'max'),
         nbr_entry=('amt', 'count'),
         grp_exp = ('amt', lambda x: x[x < 0].sum()),
         nbr_grp_exp = ('amt', lambda x: x[x < 0].count()),
         grp_inc = ('amt', lambda x: x[x > 0].sum()),
         nbr_grp_inc = ('amt', lambda x: x[x > 0].count())
        )

grp_one.describe()

In [ ]:
len(grp_lst) / x.shape[0]
# x.shape[0]

## Survival Analysis
churn event: user blocked/unfollowed

`churned`: `is_agree` is `True` and `registerd` (a.k.a. user_ts) < `last_entry`

right censored: same as`churned` is `False`

In [ ]:
# time elapsed since the last transaction entry until the
# observation cut off period tsl[0]
td['days_since'] = tsl[1] - td.last_entry

In [ ]:
td.query("registered.isna()")

calculate survival time

In [ ]:
# Is left censoring necessary? No.
(td.registered < tsl[0]).sum()

In [ ]:
# calculate start time
td['t0'] = td[['registered', 'first_entry']].min(axis=1)

In [ ]:
# calculate end time and survival_time

td['t1'] = tsl[1]
td.loc[~td.is_agree, 't1'] = td.loc[~td.is_agree, ['registered', 'last_entry']].max(axis=1, skipna=True)
td['survival_time'] = td.t1 - td.t0

In [ ]:
td[~td.is_agree & (td.t1 < tsl[1])]

In [ ]:
td[(~td.is_agree) & (td.registered < td.last_entry)]

In [ ]:
# churn
td['churned'] = ~td.is_agree & (td.registered > td.last_entry)

# is_agree not set and churned flag not set
td.loc[~td.is_agree & ~td.churned,
       ['registered', 'last_entry', 'user_tenure',
        'days_since', 'survival_time', 't1', 't0',
        'churned', 'user_id']]

In [ ]:
td[['last_entry', 'user_tenure', 'days_since', 'survival_time', ]].describe()

_**Oberservations:**_
- 50% of users churned after 7 days, 75% churned after 139 days (or ~4.6 months)
- 75% of users had not made any entries for >1146 days
- top quartile (best or most active users) had made at least one entries since 2021-12-12 (or 48~50 days)

In [ ]:
# what do you observe from the top quartile...

t0 = td.days_since.quantile(.25) # 1146 days since last entry
print('days_since =', t0.days)
(td.loc[td.days_since < t0, ['days_since', 'user_tenure', 'nbr_entry', 'last_entry']].describe())

## Segmentation by `days_since` and Tenure
- `days_since` is the number of days since the user has made the last entry
- Tenure is the number of days between the users first and last date of expense or income entry

In [ ]:
td.isna().sum()

In [ ]:
td[td.is_bot.isna()]

In [ ]:
td[td.user_id == 'U000046b3786c997220a07872c5191c37']

In [ ]:
[(x.left.round('D').days, x.right.days) for x in pd.qcut(td.days_since, q=10).unique().sort_values()]

In [ ]:
fq.xs('U000046b3786c997220a07872c5191c37')

In [ ]:
[(round(x.left), round(x.right)) for x in pd.qcut(td.user_tenure.dt.days, q=10, duplicates='drop').unique().sort_values()]
# td.info()

In [ ]:
df = td.loc[(td.days_since < td.days_since.quantile(.25)) &
            (td.user_tenure > td.user_tenure.quantile(.5)),
            ['user_id', 'days_since', 'user_tenure']].copy()
df.describe()

In [ ]:
df['days_since_decile'] = pd.qcut(df.days_since.dt.days, q=10)
df['days_since_decile'] = df['days_since_decile'].apply(lambda x: (x.right).astype('int') )  
df['user_tenure_decile'] = pd.qcut(df.user_tenure.dt.days, q=10)
df['user_tenure_decile'] = df['user_tenure_decile'].apply(lambda x: round(x.right)) 

In [ ]:
df.describe(percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9])

In [ ]:
import seaborn as sns

df_plot = df.groupby(['days_since_decile', 'user_tenure_decile'], observed=True).agg({'user_id':'count'}).reset_index()
df_plot = df_plot.pivot(index='user_tenure_decile', columns='days_since_decile', values='user_id')

# Set figure size globally
sns.set_theme(rc={'figure.figsize': (12, 6)})

_ = sns.heatmap(df_plot, annot=False, cmap='coolwarm')
df_plot.describe()

## Explore Group 

In [ ]:
#number of members in each group
nbr_mbr_grp = grp.groupby('group_id')['user_id'].nunique()

nbr_mbr_grp.agg(['min', 'max', 'mean', 'median'])

In [ ]:
nbr_mbr_grp.quantile([.25, .5, .75, .8, .9, .95, .99])

In [ ]:
# is this interesting?

cashflow.groupby(['user_id'])['amt'].sum().quantile([.1, .25, .5, .75, .9])

## Expense by Category



In [ ]:
# private entries
cashflow.query("is_group == False & is_expense == True").groupby('category') \
    .agg(n_user = ('user_id','nunique'),
         nbr_expense = ('category', 'count'),
         nbr_expense_per_user = ('user_id', lambda x: round(x.count() / x.nunique(), 2)),
         avg_amt = ('amt', lambda x: -(round(x.mean())))
        ) \
    .sort_values(by='nbr_expense',  ascending=False)

In [ ]:
# group entries
cashflow.query("is_group == True & is_expense == True").groupby('category') \
    .agg(n_user = ('user_id','nunique'),
         nbr_expense = ('category', 'count'),
         nbr_expense_per_user = ('user_id', lambda x: round(x.count() / x.nunique(), 2)),
         avg_amt = ('amt', lambda x: -(round(x.mean())))
        ) \
    .sort_values(by='nbr_expense',  ascending=False)

# Supplemental

```sql
-- consolidate expense and income entry to simplify analysis

USE zoo;

DROP TABLE IF EXISTS acc_cashflow;

CREATE TABLE acc_cashflow (
  user_id VARCHAR(100) NOT NULL,
  date DATE NOT NULL,
  amt DOUBLE NOT NULL,
  is_expense BOOLEAN,
	is_group BOOLEAN,
  group_id VARCHAR(100),
  category_id INT,
  category VARCHAR(100),
  note VARCHAR(160)
) COMMENT = 'consolidated expense and income entries'
;

-- insert personal expense
INSERT INTO acc_cashflow
SELECT
  USER_ID user_id,
  DATE(CREDTM) date,
  -AMOUNT amt,
  TRUE is_expense,
	FALSE is_group,
  NULL group_id,
  A.CATEGORY category_id,
  B.CATEGORY category,
  NOTE note
FROM zoo.ACC_USER_DETAIL A
LEFT JOIN zoo_checkchick3.ACC_CATEGORY B
ON A.CATEGORY = B.ID
;

-- insert group expense
INSERT INTO acc_cashflow
SELECT
  USER_ID user_id,
  DATE(CREDTM) date,
  -AMOUNT amt,
  TRUE is_expense,
	TRUE is_group,
  GROUP_ID group_id,
  A.CATEGORY category_id,
  B.CATEGORY category,
  NOTE note
FROM zoo_checkchick.ACC_GROUP_DETAIL A
LEFT JOIN zoo_checkchick3.ACC_CATEGORY B
ON A.CATEGORY = B.ID
;

-- insert personal income
INSERT INTO acc_cashflow
SELECT
  USER_ID user_id,
  DATE(CREDTM) date,
  AMOUNT amt,
  FALSE is_expense,
	FALSE is_group,
  NULL group_id,
  A.CATEGORY category_id,
  B.CATEGORY category,
  NOTE note
FROM zoo_checkchick2.ACC_USER_DETAIL_INCOME A
LEFT JOIN zoo_checkchick3.ACC_CATEGORY_INCOME B
ON A.CATEGORY = B.ID
;

-- insert group income
INSERT INTO acc_cashflow
SELECT
  USER_ID user_id,
  DATE(CREDTM) date,
  AMOUNT amt,
  FALSE is_expense,
	TRUE is_group,
  GROUP_ID group_id,
  A.CATEGORY category_id,
  B.CATEGORY category,
  NOTE note
FROM zoo_checkchick2.ACC_GROUP_DETAIL_INCOME A
LEFT JOIN zoo_checkchick3.ACC_CATEGORY_INCOME B
ON A.CATEGORY = B.ID
;
```